### Use full_music_data and/or the two summary data sets (with artists and years) of music characteristics, to develop measures of music similarity. Using your measure, are artists within genre more similar than artists between genres?

In [188]:
import pandas as pd
import numpy as np
import networkx as nx
import ast
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from tqdm import tqdm

In [189]:
# load data into DataFrames
artists = pd.read_csv('data_by_artist.csv')
years = pd.read_csv('data_by_year.csv')
influence = pd.read_csv('influence_data.csv')
full = pd.read_csv('full_music_data.csv')
try:
    tqdm._instances.clear()
except AttributeError:
    pass

In [190]:
def scaleAttr(df, attr):
    """scales the attribute attr of the dataframe df to go from 0 to 1"""
    floor = min(df[attr])
    span = max(df[attr]) - floor
    df[attr] = [(x-floor)/span for x in df[attr]]

In [191]:
songStats = ['danceability', 'energy', 'valence', 'tempo', 'acousticness', 'instrumentalness', 'speechiness']        
def artistSimilarity(a1, a2):
    a1Stats = artists[artists['artist_id'] == a1][songStats].to_numpy()
    a2Stats = artists[artists['artist_id'] == a2][songStats].to_numpy()
    if len(a1Stats) == 0 or len(a2Stats) == 0:
        return -1
    return float(cosine_similarity(a1Stats,a2Stats))

In [192]:
# scale tempo columns in each df
scaleAttr(full, 'tempo')
scaleAttr(artists, 'tempo')
scaleAttr(years, 'tempo')

# no dups
artists = artists.drop_duplicates(subset='artist_name', keep='last')

In [193]:
#remove songs by artists that arent in the artists table
#duplicate songs w/ more than one artist
#takes my computer about a minute to run
#can ignore the warnings about setting a copy of a slice from a dataframe
allArtists = list(artists['artist_id'])
newdict = {}

#parse id and name columns
full['artists_id'] = [int(x.split(',')[0][1:-1]) if len(x.split(',')) == 1 else [int(y) for y in x[1:-1].split(',')] for x in list(full['artists_id'])]
full['artist_names'] = [ast.literal_eval(x) if len(ast.literal_eval(x)) > 1 else ast.literal_eval(x)[0] for x in full['artist_names']]
scaleAttr(full, 'tempo')

last_id = 0
row = 0
for i in tqdm(range(len(full))): 
    thisRow = full.iloc[i,:].copy()
    artIDs = thisRow['artists_id']
    artNames = thisRow['artist_names']
    
    if type(artIDs) == int:
        if artIDs == last_id: 
            thisRow['artists_id'] = artIDs
            thisRow['artist_names'] = artNames
            newdict[row] = thisRow.to_dict()
            row+=1
            last_id = artIDs
        else:
            if artIDs in allArtists:
                thisRow['artists_id'] = artIDs
                thisRow['artist_names'] = artNames
                newdict[row] = thisRow.to_dict()
                row+=1
                last_id = artIDs
                
    else:
        for j in range(len(artIDs)):
            artID = int(artIDs[j])
            artName = str(artNames[j])
            
            if artID in allArtists:
                thisRow['artists_id'] = artID
                thisRow['artist_names'] = artName
                newdict[row] = thisRow.to_dict()
                row+=1
    i+=1

full = pd.DataFrame.from_dict(newdict,'index')

artistGenres = {} #adds song genre to full
for i in tqdm(range(len(influence))):
    row = influence.iloc[i]
    influencer = row['influencer_id']
    follower = row['follower_id']
    infGenre = row['influencer_main_genre']
    folGenre = row['follower_main_genre']
    artistGenres[influencer] = infGenre
    artistGenres[follower] = folGenre

def getGenre(row):
    artist = row['artists_id']
    if type(artist) == list:
        print(row)
    try:
        genre = artistGenres[artist]
    except KeyError:
        genre = 'N/A'
    return genre

full['genre'] = full.apply(lambda row: getGenre(row), axis = 1)
full

100%|██████████████████████████████████████████████████████████████████████████| 42770/42770 [00:07<00:00, 5644.54it/s]


,artist_names,artists_id,danceability,energy,valence,tempo,loudness,mode,key,acousticness,instrumentalness,liveness,speechiness,explicit,duration_ms,popularity,year,release_date,song_title (censored),genre
0,Fat Freddy's Drop,178301,0.600,0.365,0.131,0.532777,-13.083,0,9,0.06720,0.585000,0.0921,0.0498,0,437200,54,2005,2005,Ernie,Electronic
1,Fat Freddy's Drop,178301,0.874,0.326,0.179,0.490063,-13.302,0,11,0.01360,0.148000,0.0993,0.1310,0,581008,53,2005,2005,Wandering Eye,Electronic
2,Fat Freddy's Drop,178301,0.670,0.531,0.336,0.571037,-8.267,0,9,0.01560,0.345000,0.3060,0.0377,0,431293,55,2009,8/7/2009,The Raft,Electronic
3,Alexander O'Neal,625201,0.761,0.702,0.850,0.429237,-8.523,1,7,0.10800,0.000031,0.0935,0.0389,0,304427,34,1991,1/1/1991,All True Man,R&B;
4,Alexander O'Neal,625201,0.661,0.828,0.902,0.471455,-12.673,0,11,0.27600,0.000001,0.2870,0.0390,0,264933,37,1987,7/29/1987,(What Can I Say) To Make You Love Me,R&B;
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102913,ZZ Top,690254,0.276,0.892,0.715,0.329693,-7.035,1,11,0.40900,0.000000,0.7140,0.0893,0,115973,33,1975,4/18/1975,Jailhouse Rock - **** Remaster,Pop/Rock
102914,ZZ Top,690254,0.700,0.592,0.906,0.450025,-10.434,1,11,0.10800,0.018400,0.1180,0.0681,0,263627,32,1976,11/29/1976,It's Only Love,Pop/Rock
102915,ZZ Top,690254,0.709,0.709,0.863,0.456977,-12.023,1,2,0.11800,0.000011,0.1250,0.0370,0,158400,31,1987,1987,Balinese,Pop/Rock
102916,ZZ Top,690254,0.552,0.651,0.533,0.661835,-10.624,0,4,0.00494,0.034900,0.1320,0.2290,0,232533,43,1992,4/13/1992,La Grange - **** Remaster,Pop/Rock


In [194]:
# add edges to directed graph
# measure influence 

baseline = .9136464572663171 #from bullet 2 - average cosine similarity between a song and the era its from

idDict = {}
for i in tqdm(range(len(artists))):
    try:
        idDict[artists['artist_id'][i]] = artists['artist_name'][i]
    except KeyError:
        pass

influence = pd.read_csv('influence_data.csv')
iGraph = nx.DiGraph()

for artist in artists['artist_id']:
    iGraph.add_node(artist, weight = 0)
    
for i in tqdm(range(len(influence))):
    influencer = influence["influencer_id"][i]
    follower = influence['follower_id'][i]
    
    iGraph.add_edge(influencer,follower)    

artIDs = []
artNames = []
followers = []
grandFollowers = []
followerSimilarity = []

for i in tqdm(range(len(iGraph.nodes()))):
    node = list(iGraph.nodes())[i]
    
    children = list(iGraph.successors(node))
    nFollows = len(children)

    grandChildren = []
    similarity = 0
    for child in children:
        sim = artistSimilarity(node,child)
        if sim == -1:
            continue
        for grandChild in iGraph.successors(child):
            grandChildren.append(grandChild)
        similarity += sim
    
    artIDs.append(node)
    try:
        artNames.append(idDict[node])
    except KeyError:
        artNames.append('Unknown')
        
    followers.append(nFollows)
    grandFollowers.append(len(set(grandChildren)))
    try:
        followerSimilarity.append(similarity/nFollows)
    except ZeroDivisionError:
        followerSimilarity.append(0)
    
   
totalInfluence = pd.DataFrame({'artist_name':artNames, 'artist_id':artIDs, 'total_followers':followers,
                               '2nd_degree_followers':grandFollowers, 'follower_similarity':followerSimilarity})
totalInfluence = totalInfluence.reset_index(drop=True)
totalInfluence = totalInfluence.sort_values(by=['2nd_degree_followers'], ascending = False)
totalInfluence[:10]

100%|██████████████████████████████████████████████████████████████████████████████| 5853/5853 [01:40<00:00, 58.51it/s]


,artist_name,artist_id,total_followers,2nd_degree_followers,follower_similarity
15,The Beatles,754032,615,2690,0.938414
4,Bob Dylan,66915,389,2301,0.933637
5,The Rolling Stones,894465,319,1994,0.949717
128,Chuck Berry,120521,159,1952,0.924118
7,Elvis Presley,180228,166,1950,0.908759
167,Little Richard,824022,88,1858,0.928516
149,James Brown,128099,154,1763,0.944938
138,Jimi Hendrix,354105,201,1679,0.934165
377,Buddy Holly,538677,97,1643,0.944199
446,Bo Diddley,55128,65,1492,0.917627


In [ ]:
genreCount = full.groupby('genre').count().iloc[:,1]
genreCount = genreCount[genreCount > 1000].sort_values(ascending = False)

songStats = ['danceability', 'energy', 'valence', 'tempo', 'acousticness', 'instrumentalness', 'speechiness']        

newDict = {}
for year in tqdm(range(1960,2021)):
    inYearRange = full[full['year'] <= year+5]
    inYearRange = inYearRange[inYearRange['year'] >= year-4]
    yearAvg = inYearRange.mean()[songStats].to_dict()
    newDict[year] = yearAvg
    for genre in genres:
        allInGenre = inYearRange[inYearRange['genre'] == genre]
        rollingAvg = allInGenre.mean()[songStats].to_dict()
        newDict[str(year)+genre] = rollingAvg
genreYears = pd.DataFrame.from_dict(newDict, orient = 'index')
genreYears

 62%|███████████████████████████████████████████████████                               | 38/61 [00:33<00:13,  1.66it/s]

In [ ]:
genreDiff = {}
yearGenreDiff = {}
for i in tqdm(range(len(full))):
    row = full.iloc[i]
    try:
        genre = artistGenres[row['artists_id']]
    except KeyError:
        continue
    
    year = row['year']
    
    if genre in genres and year >= 1960:
        baseline = np.array([genreYears.loc[year].to_numpy()])
        try:
            expStats = np.array([genreYears.loc[str(year)+genre].to_numpy()])
        except KeyError:
            continue
        stats = np.array([row[songStats].to_numpy()])
        baseDiff = float(cosine_similarity(stats,baseline))
        genDiff = float(cosine_similarity(stats,expStats))
        
        try:
            genreDiff['baseline'] = np.append(genreDiff['baseline'], baseDiff)
        except KeyError:
            genreDiff['baseline'] = np.array([baseDiff])
        
        try:
            genreDiff[genre] = np.append(genreDiff[genre],genDiff)
        except KeyError:
            genreDiff[genre] = np.array([genDiff])
    
        try:
            yearGenreDiff[str(year)+genre] = np.append(genreDiff[str(year)+genre],genDiff)
        except KeyError:
            yearGenreDiff[str(year)+genre] = np.array([genDiff])
        
        try:
            yearGenreDiff[year] = np.append(yearGenreDiff[year], baseDiff)
        except KeyError:
            yearGenreDiff[year] = np.array([baseDiff])
        
for genre in list(genreDiff.keys()):
    genreDiff[genre] = np.mean(genreDiff[genre])
    

In [ ]:
import re

temp = re.compile("([0-9]+)([a-zA-Z]+)") 
newDict = {}
i=0

for key in list(yearGenreDiff.keys()):
    keyString = str(key)
    if len(keyString) == 4:
        year = keyString
        genre = 'Total'
    else:
        res = temp.match(keyString).groups()
        year = res[0]
        genre = res[1]
    newDict[i] = [year, genre, np.mean(yearGenreDiff[key])]
    i+=1
newDict

In [ ]:
allYears = pd.DataFrame.from_dict(newDict, orient = 'index', columns = ['year', 'genre', 'similarity']).sort_values('year')
dictNew = {}
for year in range(1960,2021):
    targetRows = allYears[allYears['year'] == str(year)]
    dictNew[year] = []
    for genre in list(genres.keys()):
        try:
            similarity = float(targetRows[targetRows['genre'] == genre]['similarity'])
        except TypeError:
            similarity = None
        dictNew[year].append(similarity)
genreSimilarity = pd.DataFrame.from_dict(dictNew, columns = list(genres.keys()), orient = 'index')




In [ ]:
def rollingaverage(array,rollsize):
    newarr = np.zeros(len(array))
    for i in range(len(array)):
        low = max(0,i-rollsize//2)
        high = min(len(array),i+rollsize//2)
        myarr = array[low:high]
        newval = np.mean(myarr)
        newarr[i] = newval
    return newarr


In [ ]:
a = plt.figure(figsize = (12,12))
for column in list(genreSimilarity.columns):
    toplot = rollingaverage(np.array(list(genreSimilarity[column])),15)
    plt.plot(list(range(1960,2021)),toplot)